# Purpose

Create a side-by-side image viewer with ipywidget button to make plotted region the same.

# Imports

In [1]:
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
%matplotlib widget
from IPython.display import display, display_html, HTML

import cv2
import imutils
from PIL import Image

import pandas as pd
from scipy import stats

import sys
sys.executable

'/Users/nordin/python_envs/voila_opencv/.venv/bin/python'

# Code

In [2]:
class SideBySideImageViewerBit(widgets.VBox):
    
    def __init__(self, img_l, img_r, title_left='', title_right='', vmin=0, vmax=None):
        
        super().__init__()
        output = widgets.Output()
        self.image_left = img_l
        self.image_right = img_r
        self.title_left = title_left
        self.title_right = title_right
        if vmax:
            self.vmax_initial = vmax
        else:
            self.vmax_initial = np.max(self.image_left)

        with output:
            self.fig, self.ax = plt.subplots(ncols=2, constrained_layout=True, figsize=(10, 3.5))
        self.ax_left, self.ax_right = self.ax
        
        self.axesimage_left = self.ax_left.imshow(
            self.image_left, interpolation=None, cmap="gray", vmin=vmin, vmax=self.vmax_initial
        )
        self.ax_left.set_title(self.title_left)
        self.axesimage_right = self.ax_right.imshow(
            self.image_right, interpolation=None, cmap="gray", vmin=vmin, vmax=self.vmax_initial
        )
        self.ax_right.set_title(self.title_right)
       
        style_HTML = {'description_width': 'initial'}
        self.min_max_text_left = widgets.HTML(
            value=f"{np.min(self.image_left)}, {np.max(self.image_left)}",
            placeholder="Some HTML",
            description="Left image min, max:",
            style=style_HTML,
        )
        
        self.min_max_text_right = widgets.HTML(
            value=f"{np.min(self.image_right)}, {np.max(self.image_right)}",
            placeholder="Some HTML",
            description="Right image min, max:",
            style=style_HTML,
        )

        self.vmax_slider = widgets.IntSlider(
            value=self.vmax_initial,
            min=0,
            max=1023,
            step=1,
            description="vmax",
            continuous_update=False,
            # readout_format=".4f",
        )
        
        self.xylimits_button = widgets.Button(
            description='Left image range = Right image range',
            tooltip='Make left image x,y limits the same as the right image',
            # button_style='info',
            layout=widgets.Layout(width='300px')
        )
        self.xylimits_button.on_click(self.xylimits_button_click)
        
        self.right_xylim = widgets.HTML(
            value=f"{self.ax_right.get_xlim()}, {self.ax_right.get_ylim()}",
            placeholder="Some HTML",
            description="Right image xlim, ylim:",
            style=style_HTML,
        )
        self.left_xylim = widgets.HTML(
            value=f"{self.ax_left.get_xlim()}, {self.ax_left.get_ylim()}",
            placeholder="Some HTML",
            description="Left image xlim, ylim:",
            style=style_HTML,
        )
#         self.msg = widgets.HTML(
#             value=f"msg",
#             placeholder="Some HTML",
#             description="msg",
#             style=style_HTML,
#         )

        controls = widgets.VBox(
            [
                self.vmax_slider,
                self.min_max_text_left,
                self.min_max_text_right,
                self.xylimits_button,
            ]
        )

        # observe stuff
        self.vmax_slider.observe(self.update_vmax, "value")

        # add to children
        self.children = [controls, output, self.left_xylim, self.right_xylim] #, self.msg]
        
    def update_vmax(self, change):
        self.axesimage_left.set_clim(vmax=change.new)
        self.axesimage_right.set_clim(vmax=change.new)
        self.fig.canvas.draw()
        
    def xylimits_button_click(self, value):
        # self.msg.value = f"value: {value}"
        xlim = self.ax_right.get_xlim()
        ylim = self.ax_right.get_ylim()
        self.ax_left.set_xlim(*xlim)
        self.ax_left.set_ylim(*ylim)
        self.fig.canvas.draw()
        self.update_xylim_display()
        
    def update_xylim_display(self):
        self.left_xylim.value = "({:.1f}, {:.1f}), ({:.1f}, {:.1f})".format(*self.ax_left.get_xlim(), *self.ax_left.get_ylim())
        self.right_xylim.value = "({:.1f}, {:.1f}), ({:.1f}, {:.1f})".format(*self.ax_right.get_xlim(), *self.ax_right.get_ylim())

# Example

## Create images

In [3]:
x = np.linspace(-3.2, 3.2, 1600)
y = np.linspace(-2.4, 2.4, 1200)
num_y, num_x = len(y), len(x)

img1 = np.zeros((len(y), len(x)), dtype='uint16')

size = 100
img1[num_y//2 - size//2: num_y//2 + size//2, num_x//2 - size//2: num_x//2 + size//2] = 1000

fig, ax = plt.subplots()
ax.imshow(img1, interpolation=None, cmap="gray");

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [4]:
img2 = np.zeros((len(y), len(x)), dtype='uint16')

size_y, size_x = 200, 80
img2[num_y//2 - size_y//2: num_y//2 + size_y//2, num_x//2 - size_x//2: num_x//2 + size_x//2] = 500

fig, ax = plt.subplots()
ax.imshow(img2, interpolation=None, cmap="gray");

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Side-by-side images

Procedure:

- Click on rectangle selection tool in toolbar
- Go to right image and select a region
- Click button "Left image range = Right image range"

Outcome: left image range will be the same as the right image range

In [5]:
SideBySideImageViewerBit(img1, img2, title_left='img1', title_right='img2')

SideBySideImageViewerBit(children=(VBox(children=(IntSlider(value=1000, continuous_update=False, description='…